In [1]:
import lib.data.TrainDataset as TrainDataset
import torchvision
from lib.train_util import *
import open3d as o3d
import numpy as np
from lib.geometry import index, orthogonal, perspective
from lib.model.DepthNormalizer import DepthNormalizer

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
class opt():
    dataroot = "/media/notingcode/Data/HI4D/train_test"
    projection_mode = "orthogonal"
    num_sample_inout = 5000
    num_sample_color = 0
    aug_bri = 0.0
    aug_con = 0.0
    aug_sat = 0.0
    aug_blur = 0.0
    aug_hue = 0.0
    aug_alstd = 0.0
    loadSize = 512
    yaw_list = [4, 16, 28, 40, 52, 64, 76, 88]
    num_views = 8
    random_multiview = True
    random_flip = True
    random_scale = True
    random_trans = True
    sigma = 0.005
    z_size = 200
    b_min = [-2, -1, -2]
    b_max = [2, 3, 2]
    num_stack = 4
    norm = 'group'
    hg_down = 'ave_pool'
    num_hourglass = 2
    skip_hourglass = True
    hourglass_dim = 256
    mlp_dim = [257, 1024, 512, 256, 128, 1]
    no_residual = True

In [3]:
train_dataset = TrainDataset(opt, phase='train')

/media/notingcode/Projects/PIFu/lib/data/TrainDataset.py:112: UserWarning: loadtxt: input contained no data: "/media/notingcode/Data/HI4D/train_test/val.txt"
  var_subjects = np.loadtxt(os.path.join(


In [4]:
sample = train_dataset.get_item(1)

In [5]:
sample['name']

'00011'

In [6]:
image_tensor, calib_tensor = reshape_multiview_tensors(sample['img'][None, :], sample['calib'][None, :])
sample_tensor = reshape_sample_tensor(sample['samples'][None,:], opt.num_views)
label_tensor = sample['labels'].to(device='cuda:0')

In [7]:
sample_tensor.shape

torch.Size([8, 3, 5000])

In [8]:
projected_points = perspective(sample_tensor, calib_tensor)

In [9]:
pcds = []

for idx in range(8):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(projected_points[idx].T)
    pcds.append(pcd)
    
coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(origin=pcds[0].get_center())

In [10]:
o3d.visualization.draw_geometries(pcds+[coordinate_frame])